In [123]:
#What library packagaes are to potentially be utilized
import pandas as pd
import plotly.express as px
import seaborn as sns
import numpy as np
import datapane as dp
import fbprophet as fb
import folium 
from folium.plugins import HeatMap
import geopandas as gpd
import plotly.figure_factory as ff

In [124]:
#Format DataFrame to view all content in rows and columns as well as limit values in columns to 2 digits
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [125]:
cash = pd.read_csv('cash.csv')
performance = pd.read_csv('performance.csv')
final = pd.merge(cash,performance, on=['DistrictCode','Year'])
print(final.shape)
final.head()

(213, 29)


,CO Code,DistrictCode,District,EDP 365,Current\nExpense ADA,Current\nExpense Per ADA,LEA Type,Year,AggregateLevel,CountyCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterSchool,DASS,ReportingCategory,CohortStudents,RegHSDiplomaGrad,MetUCCSUGradReq,SealOfBiliteracy,GSSealMeritDiploma,CHSPEComp,AdEdHSDiploma,SPEDCert,GEDComp,OtherTransfer,Dropout,StillEnrolled
0,4,61515,Oroville Union High,27856421.22,1981.00,14061.79769,High,2019,D,4,0,Butte,Oroville Union High,District Office,All,All,TA,513,434,112,11,99,0,0,6,0,12,46,15
1,5,61556,Bret Harte Union High,10985988.83,598.04,18369.99002,High,2019,D,5,0,Calaveras,Bret Harte Union High,District Office,All,All,TA,178,167,69,0,0,0,0,4,0,1,6,0
2,7,61630,Acalanes Union High,80123902.30,5426.46,14765.40918,High,2019,D,7,0,Contra Costa,Acalanes Union High,District Office,All,All,TA,1381,1330,1087,510,762,8,0,13,0,2,28,0
3,7,61721,Liberty Union High,90972618.34,7762.41,11719.63583,High,2019,D,7,0,Contra Costa,Liberty Union High,District Office,All,All,TA,2031,1866,937,251,371,2,19,2,25,8,42,67
4,9,61853,El Dorado Union High,77743140.76,6299.68,12340.80791,High,2019,D,9,0,El Dorado,El Dorado Union High,District Office,All,All,TA,1585,1516,875,235,529,7,0,9,1,0,25,27


In [126]:
districts = pd.read_csv('pubdistricts.csv')

In [127]:
districts.drop(districts.columns[[4, 9, 11]], axis = 1, inplace = True) 

In [128]:
districts.head(1)

,County,District,Street,City,State,Phone,Latitude,Longitude,DOCType,ZipCode,DistrictCode
0,Contra Costa,Acalanes Union High,1212 Pleasant Hill Road,Lafayette,CA,(925) 280-3900,37.905816,-122.0998,High School District,94549,61630


In [129]:
final_dc = set(final.DistrictCode.unique())

In [130]:
districts_dc = set(districts.DistrictCode.unique())

In [131]:
final_district_dc = final_dc.intersection(districts_dc)

In [132]:
final_2 = final[final['DistrictCode'].isin(final_district_dc)]
final_2.shape

(213, 29)

In [133]:
districts_2 = districts[districts['DistrictCode'].isin(final_district_dc)]
districts_2.shape

(71, 11)

In [134]:
test = districts_2.merge(final_2, on='DistrictCode')

In [135]:
test.shape

(213, 39)

In [136]:
eda = test.copy()

In [137]:
t = eda[['Latitude', 'Longitude']].values
np.sort(t)
coordinates = np.array(t).tolist()
eda['Coordinates'] = coordinates
eda['Current\nExpense Per ADA'] = round(eda['Current\nExpense Per ADA'], 2)
# eda['Current\nExpense Per ADA'] = eda['Current\nExpense Per ADA'].map('${:,.2f}'.format)
# eda['EDP 365'] = eda['EDP 365'].map('${:,.2f}'.format)
# eda['Current\nExpense ADA'] = eda['Current\nExpense ADA'].map('${:,.2f}'.format)

In [138]:
#Rename and drop columns in DataFrame
eda = eda.rename(columns = {'District_x':'District'})
eda = eda.rename(columns={'EDP 365': 'edp365', 'Current\nExpense ADA': 'current_n_expense_ada'})
eda = eda.drop(['District_y','DistrictName', 'CountyName', 'Phone'],axis=1)

In [139]:
#Rearrange Column Order
cols = list(eda.columns)
cols[35], cols[7] = cols[7], cols[35]
eda = eda[cols]

In [141]:
#Create new DataFrame for original DataFrame column names: helps identify index number faster
cols
from pandas import DataFrame
eda_cols = DataFrame (cols,columns=['EDA_Column_Names'])
eda_cols
eda['GradRate'] = round(eda['RegHSDiplomaGrad'] / eda['CohortStudents'] * 100, 2) 
eda['CollegeReadinessRate'] = round(eda['MetUCCSUGradReq'] / eda['CohortStudents'] * 100, 2) 
eda['DropOutRate'] = round(eda['Dropout'] / eda['CohortStudents'] * 100, 2) 
eda['edp365'] = round(eda['edp365'], 2)
eda.to_csv('eda.csv', index=False)

In [142]:
la = eda[eda['County'] == 'Los Angeles']
la

,County,District,Street,City,State,Latitude,Longitude,Coordinates,ZipCode,DistrictCode,CO Code,edp365,current_n_expense_ada,Current\nExpense Per ADA,LEA Type,Year,AggregateLevel,CountyCode,SchoolCode,SchoolName,CharterSchool,DASS,ReportingCategory,CohortStudents,RegHSDiplomaGrad,MetUCCSUGradReq,SealOfBiliteracy,GSSealMeritDiploma,CHSPEComp,AdEdHSDiploma,SPEDCert,GEDComp,OtherTransfer,Dropout,StillEnrolled,DOCType,GradRate,CollegeReadinessRate,DropOutRate
9,Los Angeles,Antelope Valley Union High,44811 N. Sierra Highway,Lancaster,CA,34.696790,-118.13675,"[34.69679, -118.13675]",93534,64246,19,2.709913e+08,19425.97,13949.95,High,2019,D,19,0,District Office,All,All,TA,5433,4284,1730,234,860,1,0,12,0,36,599,501,High School District,78.85,31.84,11.03
10,Los Angeles,Antelope Valley Union High,44811 N. Sierra Highway,Lancaster,CA,34.696790,-118.13675,"[34.69679, -118.13675]",93534,64246,19,2.561844e+08,19360.45,13232.36,High,2018,D,19,0,District Office,All,All,TA,5541,4293,1436,252,409,4,0,13,1,66,605,559,High School District,77.48,25.92,10.92
11,Los Angeles,Antelope Valley Union High,44811 N. Sierra Highway,Lancaster,CA,34.696790,-118.13675,"[34.69679, -118.13675]",93534,64246,19,2.380238e+08,19881.81,11971.94,High,2017,D,19,0,District Office,All,All,TA,5784,4260,1511,239,432,6,0,35,3,81,663,736,High School District,73.65,26.12,11.46
21,Los Angeles,Centinela Valley Union High,14901 South Inglewood Avenue,Lawndale,CA,33.896373,-118.36136,"[33.896373, -118.36136]",90260,64352,19,9.585969e+07,5935.08,16151.37,High,2019,D,19,0,District Office,All,All,TA,3639,1495,636,187,186,0,0,5,0,17,2064,58,High School District,41.08,17.48,56.72
22,Los Angeles,Centinela Valley Union High,14901 South Inglewood Avenue,Lawndale,CA,33.896373,-118.36136,"[33.896373, -118.36136]",90260,64352,19,8.438654e+07,5964.62,14147.85,High,2018,D,19,0,District Office,All,All,TA,2558,1493,583,146,182,2,0,0,0,11,991,61,High School District,58.37,22.79,38.74
23,Los Angeles,Centinela Valley Union High,14901 South Inglewood Avenue,Lawndale,CA,33.896373,-118.36136,"[33.896373, -118.36136]",90260,64352,19,8.034282e+07,6025.60,13333.58,High,2017,D,19,0,District Office,All,All,TA,2248,1414,563,188,164,0,0,3,0,16,679,136,High School District,62.90,25.04,30.20
51,Los Angeles,El Monte Union High,3537 Johnson Avenue,El Monte,CA,34.071779,-118.03059,"[34.071779, -118.03058999999999]",91731,64519,19,1.213146e+08,8130.07,14921.72,High,2019,D,19,0,District Office,All,All,TA,2217,1861,908,276,402,4,0,23,0,19,244,66,High School District,83.94,40.96,11.01
52,Los Angeles,El Monte Union High,3537 Johnson Avenue,El Monte,CA,34.071779,-118.03059,"[34.071779, -118.03058999999999]",91731,64519,19,1.192204e+08,8376.43,14232.84,High,2018,D,19,0,District Office,All,All,TA,2302,1958,991,311,414,4,1,25,0,120,131,63,High School District,85.06,43.05,5.69
53,Los Angeles,El Monte Union High,3537 Johnson Avenue,El Monte,CA,34.071779,-118.03059,"[34.071779, -118.03058999999999]",91731,64519,19,1.112083e+08,8522.02,13049.52,High,2017,D,19,0,District Office,All,All,TA,2177,1821,836,304,383,0,0,23,0,72,182,79,High School District,83.65,38.40,8.36
204,Los Angeles,Whittier Union High,9401 South Painter Avenue,Whittier,CA,33.952059,-118.04429,"[33.952059000000006, -118.04428999999999]",90605,65128,19,1.582461e+08,10868.30,14560.33,High,2019,D,19,0,District Office,All,All,TA,2850,2615,1530,434,806,3,3,42,0,41,69,77,High School District,91.75,53.68,2.42


In [143]:
import plotly.express as px

fig = px.scatter_mapbox(eda, lat="Latitude", lon="Longitude", color="edp365", size="MetUCCSUGradReq",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=7,
                  mapbox_style="carto-positron")

fig.update_layout(
        title={
            'text': "Distribution of Funding Measured as Size of Students Who Met UC/CSU GradReq",
            'x':0.5,
            'xanchor': 'center'})

fig.update_layout(
    template="plotly_dark"
) 

fig.show()

In [144]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("EDP 365 v. Reg HS Diploma Grad", "EDP 365 v. Met UC/CSU GradReq", "EDP 365 v. Special Ed. Cert.", "EDP 365 v. Dropout Rate"))

fig.add_trace(go.Scatter(x=eda['edp365'], y=eda['RegHSDiplomaGrad']), row=1, col=1)

fig.add_trace(go.Scatter(x=eda['edp365'], y=eda['MetUCCSUGradReq']), row=1, col=2)

fig.add_trace(go.Scatter(x=eda['edp365'], y=eda['SPEDCert']), row=2, col=1)

fig.add_trace(go.Scatter(x=eda['edp365'], y=eda['Dropout']), row=2, col=2)

# fig.update_xaxes(title_text="edp365", row=1, col=1)
# fig.update_xaxes(title_text="edp365", row=1, col=2)
# fig.update_xaxes(title_text="edp365", row=2, col=1)
# fig.update_xaxes(title_text="edp365", row=2, col=2)

# Update yaxis properties
# fig.update_yaxes(title_text="# of RegHSDiploma Grad", row=1, col=1)
# fig.update_yaxes(title_text="# of Student who Met UCCSUGradReq", row=1, col=2)
# fig.update_yaxes(title_text="# of Special Education Certificates", row=2, col=1)
# fig.update_yaxes(title_text="# of Dropouts", row=2, col=2)

fig.update_layout(
        title={
            'text': "Relationships in Funding: EDP 365",
            'x':0.5,
            'xanchor': 'center'})

fig.update_layout(
    template="plotly_dark"
)

fig.show() 

In [270]:
import plotly.express as px

fig = px.bar(eda, x="District", y="edp365", color="RegHSDiplomaGrad", template='plotly_dark')

fig.update_layout(
        title={
            'text': "District Expenses v. High School Graduates",
            'x':0.5,
            'xanchor': 'center'})

fig.show()

In [147]:
fig = px.scatter_matrix(eda, 
                        dimensions=['edp365', 'RegHSDiplomaGrad', 'MetUCCSUGradReq', 'SPEDCert'],
                        color='District',
                        template='plotly_dark',
                       width=900,
                       height=600)

fig.update_layout(
        title_text="California: Observation of The Allocation of High School Funding v. Performance",
        title_xanchor="center",
        title_font=dict(size=20),
        title_x=0.5
    
    )

fig.show()

In [148]:
#create a map
this_map = folium.Map(prefer_canvas=True, tiles="Stamen Toner")

def coordinate(locate):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[locate.Latitude, locate.Longitude],
                        radius=5,
                        tooltip = locate.City,
                        color='#3186cc',
                        fill=True,
                        popup = locate.District,
                        fill_color='#3186cc').add_to(this_map)

#use df.apply(,axis=1) to iterate through every row in your dataframe
eda.apply(coordinate, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

this_map

In [149]:
eda_cols

,EDA_Column_Names
0,County
1,District
2,Street
3,City
4,State
5,Latitude
6,Longitude
7,Coordinates
8,ZipCode
9,DistrictCode


In [ ]:
# # px.set_mapbox_access_token(mapbox_access_token)
# fig = px.scatter_mapbox(eda, lat="Latitude", lon="Longitude", 
#                         color="City", 
#                         size='EDP 365', size_max=10, 
#                         category_orders={'County':list(eda.County.unique())},
#                         color_discrete_sequence=['#2b83ba', '#fdae61'],
# #                         hover_data=1,
# #                         hover_name='hour',
# #                         text='hour',
                        
#                         zoom=4,
#                         #center=dict(lon=105, lat=38),
#                         width=1000, height=700
#                         #title='modified scatter size'
#                        )
# fig.show()

In [ ]:
# mapbox_access_token =  'pk.eyJ1IjoicmJiZWxsODciLCJhIjoiY2pyODh3d2F5MDRiMTQzczdzajZtdHE0dSJ9.kJYzMbEc4blldGKGeS2Vbw'
# px.set_mapbox_access_token(mapbox_access_token)

# fig = ff.create_hexbin_mapbox(
#     data_frame=eda, lat="Latitude", lon="Longitude",
#     nx_hexagon=10, opacity=0.9, labels={"color": 'Current\nExpense Per ADA'},
# )
# fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
# fig.show()

In [150]:
# play with the color: continous/numerical
# px.set_mapbox_access_token(mapbox_access_token)
mapbox_access_token =  'pk.eyJ1IjoicmJiZWxsODciLCJhIjoiY2pyODh3d2F5MDRiMTQzczdzajZtdHE0dSJ9.kJYzMbEc4blldGKGeS2Vbw'
px.set_mapbox_access_token(mapbox_access_token)
fig = px.scatter_mapbox(eda, lat="Latitude", lon="Longitude", color="County", 
                         size="edp365",
                        color_continuous_scale=px.colors.cyclical.Twilight, 
                        
                        zoom=4.5,
                        #center= (lat=37, lon=-122),
                       width=700,
                        height=700,)
fig.show()

In [151]:
mapbox_access_token =  'pk.eyJ1IjoicmJiZWxsODciLCJhIjoiY2pyODh3d2F5MDRiMTQzczdzajZtdHE0dSJ9.kJYzMbEc4blldGKGeS2Vbw'
px.set_mapbox_access_token(mapbox_access_token)
fig = px.scatter_mapbox(eda, lat="Latitude", lon="Longitude", color="City", zoom=4,
                      mapbox_style='open-street-map',
                       )
fig.show()

In [ ]:
import statsmodels.formula.api as sm

model3 = 'edp365~District'
lm3   = sm.ols(formula = model3, data = eda).fit()
lm3.summary()

In [ ]:
model4 = 'edp365~RegHSDiplomaGrad +  District'
lm4   = sm.ols(formula = model4, data = eda).fit()
lm4.summary()

In [ ]:
model5 = 'edp365~District +  MetUCCSUGradReq'
lm5   = sm.ols(formula = model5, data = eda).fit()
lm5.summary()

In [ ]:
model6 = 'edp365~District +  SPEDCert'
lm6   = sm.ols(formula = model6, data = eda).fit()
lm6.summary()

In [ ]:
model7 = 'edp365~District +  DropOutRate'
lm7   = sm.ols(formula = model7, data = eda).fit()
lm7.summary()

In [ ]:
model8 = 'edp365~District +  RegHSDiplomaGrad + MetUCCSUGradReq + SPEDCert + DropOutRate'
lm8   = sm.ols(formula = model8, data = eda).fit()
lm8.summary()

In [234]:
import plotly.express as px
import numpy as np

fig = px.treemap(eda, path=[px.Constant('State'), 'District'], values='CohortStudents',
                  color='edp365', hover_data=['CohortStudents'])

fig.update_layout(
    title="Funding Analysis at District Level",
    font=dict(
        family="Times New Roman",
        size=16,
        color="white"
    )
)

fig.update_layout(
    template="plotly_dark"
)

fig.update_layout(
        title={
            
            'x':0.5,
            'xanchor': 'center'})

fig.show()

In [241]:
import plotly.express as px
import numpy as np

fig = px.treemap(eda, path=[px.Constant('State'), 'District'], values='edp365',
                  color='MetUCCSUGradReq', hover_data=['CohortStudents'])

fig.update_layout(
    title="Met UC/ CSU Graduation Requirement at District Level",
    font=dict(
        family="Times New Roman",
        size=16,
        color="white"
    )
)

fig.update_layout(
    template="plotly_dark"
)

fig.update_layout(
        title={
            
            'x':0.5,
            'xanchor': 'center'})

fig.show()
# plotly.offline.plot(fig, filename='eda.html')

In [156]:
import plotly
import plotly.express as px
import numpy as np

fig = px.treemap(eda, path=[px.Constant('State'), 'County', 'District', 'City'], values='CohortStudents',
                  color='edp365', hover_data=['CohortStudents'],
                 title='Funding Analysis at County and District Levels')

fig.update_layout(
    title="Funding Analysis at County and District Levels",
    font=dict(
        family="Times New Roman",
        size=16,
        color="white"
    )
)

fig.update_layout(
    template="plotly_dark"
)

fig.show() 

#plotly.offline.plot(fig, filename='eda.html')

In [261]:
a = eda.groupby(['District'])[['edp365']].agg('sum')
a = a.sort_values(by='edp365', ascending=True)
bottom = a.head(5)
bottom = bottom.reset_index(drop=False)
bottom.columns
bottom

,District,edp365
0,Dunsmuir Joint Union High,5241050.00
1,Julian Union High,8088418.23
2,East Nicolaus Joint Union High,10201512.39
3,Summerville Union High,18678124.06
4,Le Grand Union High,20863269.89


In [228]:
b = a.sort_values(by='edp365', ascending=False)
top = b.head(5)
top = top.reset_index(drop=False)
top.columns
top

,District,edp365
0,Kern High,1.397554e+09
1,Sweetwater Union High,1.393274e+09
2,Anaheim Union High,1.045749e+09
3,Chaffey Joint Union High,8.286607e+08
4,East Side Union High,8.094959e+08


In [248]:
fig = px.bar(top, x='District', y='edp365',height=500, text='edp365')

fig.update_layout(
    #title="Top 5 Schools With Most Funding From 2017-2019",
    font=dict(
        family="Times New Roman",
        size=16,
        color="white"
    )
)


fig.update_layout(
    template="plotly_dark"
)

fig.update_layout(
        title={
            'text': "Top 5 Districts With Most Funding From 2017-2019",
            'x':0.5,
            'xanchor': 'center'})

fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')


fig.show() 

In [247]:
x = bottom['District']
y = bottom['edp365']

fig = px.bar(bottom, x='District', y='edp365',height=500, text='edp365')

fig.update_layout(
    font=dict(
        family="Times New Roman",
        size=16,
        color="white"
    )
)

fig.update_layout(
    template="plotly_dark"
)

fig.update_layout(
        title={
            'text': "Bottom 5 Districts With Least Funding From 2017-2019",
            'x':0.5,
            'xanchor': 'center'})

fig.update_traces(texttemplate='%{text:.2s}', textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.show() 

In [263]:
import plotly.graph_objects as go
import plotly.figure_factory as ff

fig = ff.create_table(round(top,2), height_constant=100)
# fig1 = ff.create_table(bottom, height_constant=60)

trace1 = go.Scatter(x=top['District'], y=top['edp365'],
                    marker=dict(color='#0099ff'),
                    name='Top 5 Funded Districts',
                    xaxis='x2', yaxis='y2')
# trace2 = go.Scatter(x=bottom['District'], y=bottom['edp365'],
#                     marker=dict(color='#404040'),
#                     name='Goals Against<br>Per Game',
#                     xaxis='x2', yaxis='y2')

fig.add_traces([trace1])

# initialize xaxis2 and yaxis2
fig['layout']['xaxis2'] = {}
fig['layout']['yaxis2'] = {}

# Edit layout for subplots
fig.layout.xaxis.update({'domain': [0, .5]})
fig.layout.xaxis2.update({'domain': [0.6, 1.]})

# The graph's yaxis MUST BE anchored to the graph's xaxis
fig.layout.yaxis2.update({'anchor': 'x2'})
fig.layout.yaxis2.update({'title': 'Funding'})

# Update the margins to add a title and see graph x-labels.
fig.layout.margin.update({'t':50, 'b':100})
fig.layout.update({'title': 'Top 5 Funded Districts 2017-2019'})

fig.update_layout(
    template="plotly_dark"
)

fig.show()
import chart_studio.tools as tls

# tls.get_embed('https://plotly.com/~chris/1638')

In [ ]:
import plotly.express as px
import numpy as np

fig = px.sunburst(eda, path=[px.Constant('State'), 'County', 'District', 'City'], values='CohortStudents',
                  color='edp365', hover_data=['CohortStudents'])
fig.show()

In [ ]:
import plotly.express as px
import numpy as np

fig = px.sunburst(eda, path=[px.Constant('State'), 'County', 'District', 'City'], 
                  values='edp365',
                  color='CohortStudents', hover_data=['CohortStudents'])
fig.show()

In [ ]:
import plotly.express as px
import numpy as np

fig = px.treemap(eda, path=[px.Constant('State'), 'County', 'District', 'City'], 
                  values='edp365',
                  color='CohortStudents', hover_data=['CohortStudents'])
fig.show()